In [1]:
import os
import os.path
import subprocess
import getopt
import sys
import math
# from numpy import mean
# from numpy import std
from scipy.stats import pearsonr
import operator
import time
import glob
import numpy as np

def runCMD(cmd):
    val = subprocess.Popen(cmd, shell=True).wait()
    if val == 0:
        pass
    else:
        print ('command failed')
        print (cmd)
        sys.exit(1)


In [2]:
def runMR(id1,id2):
    mranks = ranka[id1][id2] * ranka[id2][id1]
    mr = np.round(np.sqrt(mranks), 3)

    mra[id1][id2] = mr


In [10]:
infile = '/depot/jwisecav/data/jwisecav/repo_testing/test.matrix'
outdir = '/depot/jwisecav/data/jwisecav/repo_testing/test_mr_6'
skipfile = '/depot/jwisecav/data/jwisecav/repo_testing/exclude_genes.txt'
threads = 4
t = time.time()
tmpdir = '/tmp/' + str(t)

if os.path.exists(tmpdir) == False:
    cmd = 'mkdir ' + tmpdir
    runCMD(cmd)
    print('Creating output directory:', tmpdir, '...\n')
else:
    print('Writing to existing output directory:', tmpdir, '...\n')
    
if os.path.exists(outdir) == False:
    cmd = 'mkdir ' + outdir
    runCMD(cmd)
    print('Creating output directory:', outdir, '...\n')
else:
    print('Writing to existing output directory:', outdir, '...\n')


Creating output directory: /tmp/1569287443.8788483 ...

Writing to existing output directory: /depot/jwisecav/data/jwisecav/repo_testing/test_mr_6 ...



In [11]:
tmpdir = '/tmp/1569279299.0313113'

In [12]:
expDict = {}
header = 0

fi = open(infile)
print('Reading in gene expression values from', infile, '...\n')
for line in fi:
    if header == 0:
        header += 1
        continue
    
    exp = line.rstrip().split('\t')
    gene = exp.pop(0)
    exp = [float(i) for i in exp]
    #print(gene, len(col))

    expDict[gene] = exp
    
fi.close()

try:
    skipfile
except NameError:
    print('Skipping 0 genes...\n')
else:
    fi = open(skipfile)
    skipcount = 0
    for gene in fi:
        gene = gene.rstrip()
        if gene in expDict:
            del expDict[gene]
            skipcount += 1
    print('Skipping', skipcount, 'genes...\n')

    fi.close()

print('Creating gene lookup tables...\n')
idlookup = {}
genelookup = {}
gid = 0
for gene in expDict:
    print(gene,gid)
    idlookup[gene] = gid
    genelookup[gid] = gene
    gid = gid + 1 

for gene in idlookup:
    gid = idlookup[gene]
    expDict[gid] = expDict.pop(gene)

#print(expDict.keys())

Reading in gene expression values from /depot/jwisecav/data/jwisecav/repo_testing/test.matrix ...

Skipping 2 genes...

Creating gene lookup tables...

Sevir.1G000100 0
Sevir.1G000200 1
Sevir.1G000300 2
Sevir.1G000400 3
Sevir.1G000501 4
Sevir.1G000600 5
Sevir.1G000650 6
Sevir.1G001000 7
Sevir.1G001100 8
Sevir.1G001200 9
Sevir.1G001300 10
Sevir.1G001400 11
Sevir.1G001500 12
Sevir.1G001600 13
Sevir.1G001700 14
Sevir.1G001800 15
Sevir.1G001900 16
Sevir.1G002000 17
Sevir.1G002100 18
Sevir.1G002200 19
Sevir.1G002300 20
Sevir.1G002400 21
Sevir.1G002500 22
Sevir.1G002600 23
Sevir.1G002800 24
Sevir.1G002900 25


In [18]:
subDict = {}
num_genes = len(expDict)
subcount = math.ceil(num_genes/threads)

gct = 0
tct = 1
for gid in expDict:
    gct += 1 
    
    if gct == 1:
        subDict[tct] = []
    
    if gct > subcount:
        gct = 1
        tct += 1
        subDict[tct] = []
        
    subDict[tct].append(gid)

sum = 0
for tct in subDict:
    sublen = len(list(subDict[tct]))
    sum = sum + sublen
if num_genes != sum:
    print('WARNING')
    print('total genes:', num_genes)
    print('total split genes:', sum)
    
#print(subDict[4])

[21, 22, 23, 24, 25]


In [7]:
numgenes = len(idlookup)
mra = np.zeros(shape=(numgenes,numgenes))
ranka = np.zeros(shape=(numgenes,numgenes))

for gid1 in range(0, numgenes):
    for gid2 in range(0, numgenes):
        corr,_ = pearsonr(expDict[gid1], expDict[gid2])
        #print(gene1,gene2,corr)
        mra[gid1][gid2] = corr
        mra[gid2][gid1] = corr

for gid1 in range(0, numgenes):    
    count = 0
    d = dict(enumerate(mra[gid1,]))
    sorted_d = sorted(d.items(), key=operator.itemgetter(1), reverse = True)
    
    for gc in sorted_d:
        gid2 = gc[0]
        corr = gc[1]    
        ranka[gid1][gid2] = count
        count += 1

for gid1 in range(0, numgenes):
    for gid2 in range(0, numgenes):
        if gid2 > gid1:
            runMR(gid1,gid2)


In [8]:
for gid1 in range(0, numgenes):
    mrDict = {}
    pccDict = {}

    for gid2 in range(0, numgenes):
        if gid1 == gid2:
            continue

        mr = ''
        pcc = ''
        if gid2 > gid1:
            mr = mra[gid1][gid2]
            pcc = mra[gid2][gid1]        
        elif gid1 > gid2:
            pcc = mra[gid1][gid2]
            mr = mra[gid2][gid1]
        
        mrDict[gid2] = mr
        pccDict[gid2] = pcc
    
    gene = genelookup[gid1]
    outfile = outdir + '/' + gene
    fo = open(outfile, 'w')

    sorted_d = sorted(mrDict.items(), key=operator.itemgetter(1), reverse = False)
    for gm in sorted_d:
        [gid2, mr] = gm
        gene2 = genelookup[int(gid2)]

        line = gene2 + '\t' + str(np.round(mr,3)) + '\t' + str(pccDict[gid2]) + '\n'
        fo.write(line)
    
    fo.close()



In [9]:
! cat {outdir}/Sevir.1G001800

Sevir.1G001000	2.236	0.33424565444399756
Sevir.1G001900	3.464	0.31059174658961697
Sevir.1G001500	4.899	0.24316493561426894
Sevir.1G002100	4.899	0.18714644403051597
Sevir.1G000501	6.708	0.033864351836590686
Sevir.1G001200	7.746	-0.018905242979886508
Sevir.1G002000	7.746	0.13985193705690513
Sevir.1G002200	7.937	0.06741019136156762
Sevir.1G000100	8.062	0.0015885192526926151
Sevir.1G002400	8.485	0.1297360913039204
Sevir.1G000400	9.798	0.00826162581407579
Sevir.1G002900	10.583	0.051623190395410454
Sevir.1G001400	11.402	0.031097042105359157
Sevir.1G002600	13.266	0.014295633752603084
Sevir.1G001300	16.31	-0.006962950897570396
Sevir.1G000300	16.492	-0.01893308007559849
Sevir.1G000600	17.972	-0.0271413609034647
Sevir.1G001700	19.0	-0.035651246062497144
Sevir.1G000650	19.9	-0.031124947771508173
Sevir.1G001600	19.975	-0.059396104883355255
Sevir.1G002300	20.976	-0.04386160052531114
Sevir.1G002500	21.448	-0.11305303331108822
Sevir.1G001100	22.978	-0.11854434806371451
Sevir.1G000200	23.452	-0.172575

In [19]:
format = "%(asctime)s: %(message)s"
print(format)

%(asctime)s: %(message)s
